In [16]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

In [17]:
def remove(text):
    return text[:]

def r_dup(sent):
    words = sent.split() 
    res = [] 
    for word in words: 
        # If condition is used to store unique string  
        # in another list 'k'  
        if (sent.count(word)>1 and (word not in res)or sent.count(word)==1): 
            res.append(word) 
    return ' '.join(res)

def text_process(mess):
    
    mess = mess.replace('/', ' ')
    mess = mess.replace('(',' ')
    mess = mess.replace(')',' ')
    mess = mess.replace('-',' ')
    
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    nopunc = [word for word in nopunc.split() if word.lower() not in stopwords.words('russian')]
    return ' '.join(nopunc)

def lemmatize(text):
    res = list()
    words = text.split() # разбиваем текст на слова
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
        
    #removing numbers    
    res = [''.join(x for x in i if x.isalpha()) for i in res] 
    while '' in res:
        
        res.remove('')     
    
    return ' '.join(res)

def pos_tag(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.tag.POS)
    return res      

In [18]:
app = Flask(__name__)
model = pickle.load(open('model_LR.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = request.form.values()
    final_features = int_features
    prediction = model.predict(final_features)

    output = round(prediction[0], 2)

    return render_template('index.html', prediction_text='Prediction {}'.format(output))

@app.route('/predict_api',methods=['POST'])
def predict_api():
    '''
    For direct API calls trought request
    '''
    res = request.get_json(force=True)
    data = json.loads(res.text)
    data = json.dumps(data)
    data = data.apply(r_dup)
    data = data.apply(text_process)
    data = data.apply(lemmatize)
    prediction = model.predict_proba(data)

    output = prediction[0]
    return jsonify(output)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1